# Evaluate

In [0]:
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load the data from the CSV file
data_path = 'data/tweets_encryption_test.csv'
data = pd.read_csv(data_path)

# Convert the data to a DataFrame
tweets_eval = dataiku.Dataset("tweets_eval")
eval_df = tweets_eval.get_dataframe()

X_eval, y_eval, _ = preprocess_data(eval_df, tfidf)

eval_prediction_results_df = eval_df.copy()

eval_prediction_results_df['predicted'] = lr_model.predict(X_eval)
eval_prediction_results_df['probability'] = lr_model.predict_proba(X_eval)[:, 1]
eval_prediction_results_df['correct_prediction'] = results_df['label'] == results_df['predicted']

# Calculate evaluation metrics
accuracy = accuracy_score(y_eval, results_df['predicted'])
precision = precision_score(y_eval, results_df['predicted'], average='weighted')
recall = recall_score(y_eval, results_df['predicted'], average='weighted')
f1 = f1_score(y_eval, results_df['predicted'], average='weighted')
roc_auc = roc_auc_score(y_eval, lr_model.predict_proba(X_eval), multi_class='ovr', average='weighted')

# Print the evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"ROC AUC: {roc_auc}")

eval_metrics_df = pd.DataFrame({
    'Mean Accuracy': [accuracy],
    'Mean Precision': [precision],
    'Mean Recall': [recall],
    'Mean F1 Score': [f1],
    'Mean ROC AUC': [roc_auc],
    'Model Name': ['Logistic Regression'],
    'Date and Time': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')]
})


# Create output datasets

In [0]:
# Recipe outputs
eval_prediction_results = dataiku.Dataset("eval_prediction_results")
eval_prediction_results.write_with_schema(eval_prediction_results_df)

eval_metrics = dataiku.Dataset("eval_metrics")
eval_metrics.write_with_schema(eval_metrics_df)
